# Advance Orders

In [1]:
import os
os.chdir(os.path.pardir)
os.chdir(os.path.pardir)

In [2]:
from src.market import MarketBase
from src.market_actor.stub import MarketActorStub
from src.market_listener.stub import MarketListenerStub

from pyutils.events import wait_for

In [3]:
market_listener = MarketListenerStub()
market = MarketBase("AAPL", "USD", market_listener)
market_actor = MarketActorStub(transact_flat_cost=0, transact_min_cost=0, transact_var_cost_rate_percent=0.2)

## 1. Stop Loss Clases

In [4]:
from src.advance_order.position_closing.stop_loss import StopLoss, TrailingStopLoss, TrailingPercentStopLoss

### 1.1. StopLoss Class

#### StopLoss Trigger and Execution

In [5]:
# Execute buy order
market_listener.set_current_price(1000)
buy_order = market_actor.make_order(market, 1000, 1)
market_actor.execute_order(buy_order)
wait_for(buy_order.has_filled)

open_position = buy_order.filled_position
open_position

Submitted buy order of 1.00 AAPL       at 1000.00
Filled buy order of 1.00 AAPL       at 1000.00


{'AAPL': 0.998, 'USD': -1000.0}

In [6]:
# Create StopLoss
stop_loss_price = 980
stop_loss = StopLoss.make_advance_order(market, market_actor, open_position,
        stop_loss_price=stop_loss_price, auto_assimilate=True)

market_actor.activate_advance_order(stop_loss)

In [7]:
market_listener.set_current_price(1100)

In [8]:
market_listener.set_current_price(980)

Submitted sell order of 1.00 AAPL       at 980.00
Filled sell order of 1.00 AAPL       at 980.00


In [9]:
open_position.closed

True

#### StopLoss Cancellation

In [10]:
# Execute buy order
market_listener.set_current_price(1000)
buy_order = market_actor.make_order(market, 1000, 1)
market_actor.execute_order(buy_order)
wait_for(buy_order.has_filled)

open_position = buy_order.filled_position
open_position

Submitted buy order of 1.00 AAPL       at 1000.00
Filled buy order of 1.00 AAPL       at 1000.00


{'AAPL': 0.998, 'USD': -1000.0}

In [11]:
# Create StopLoss
stop_loss_price = 980
stop_loss = StopLoss.make_advance_order(market, market_actor, open_position,
        stop_loss_price=stop_loss_price, auto_assimilate=True)

In [12]:
# Activate StopLoss and cancel
market_actor.activate_advance_order(stop_loss)

market_listener.set_current_price(1100)
stop_loss.cancel_order()

In [13]:
open_position.closed

False

### 2. TrailingStopLoss Class

In [14]:
# Execute short order
market_listener.set_current_price(1000)
short_order = market_actor.make_order(market, 1000, -1)
market_actor.execute_order(short_order)
wait_for(short_order.has_filled)

open_position = short_order.filled_position
open_position

Submitted sell order of 1.00 AAPL       at 1000.00
Filled sell order of 1.00 AAPL       at 1000.00


{'AAPL': -1, 'USD': 998.0}

In [15]:
# Create TrailingStopLoss
trailing_stop_loss = TrailingStopLoss.make_advance_order(market, market_actor, open_position,
        trailing_gap=50, auto_assimilate=True)

market_actor.activate_advance_order(trailing_stop_loss)

In [16]:
market_listener.set_current_price(900)

In [17]:
market_listener.set_current_price(950)

Submitted buy order of 1.00 AAPL       at 950.00
Filled buy order of 1.00 AAPL       at 950.00


In [18]:
open_position.closed

True

### 3. TrailingPercentStopLoss Class

In [20]:
# Execute buy order
market_listener.set_current_price(1000)
buy_order = market_actor.make_order(market, 1000, 1)
market_actor.execute_order(buy_order)
wait_for(buy_order.has_filled)

open_position = buy_order.filled_position
open_position

Submitted buy order of 1.00 AAPL       at 1000.00
Filled buy order of 1.00 AAPL       at 1000.00


{'AAPL': 0.998, 'USD': -1000.0}

In [21]:
# Create TrailingPercentStopLoss
trailing_stop_loss = TrailingPercentStopLoss.make_advance_order(market, market_actor, open_position,
        trailing_gap_percent=1.5, auto_assimilate=True)

market_actor.activate_advance_order(trailing_stop_loss)

In [22]:
market_listener.set_current_price(1100)

In [23]:
market_listener.set_current_price(1050)

Submitted sell order of 1.00 AAPL       at 1050.00
Filled sell order of 1.00 AAPL       at 1050.00


In [24]:
open_position.closed

True